<a href="https://colab.research.google.com/github/zlzonhenu/RNN-tutorial/blob/master/longseq_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
torch.manual_seed(0)

In [0]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [4]:
#make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_set)
print(char_dic)

['u', 's', 'e', ',', 'c', ' ', 'y', 'a', 'p', 'o', 'd', 'f', 'l', 'w', 'r', 'h', "'", 'g', 'm', 't', 'k', '.', 'n', 'b', 'i']
{'u': 0, 's': 1, 'e': 2, ',': 3, 'c': 4, ' ': 5, 'y': 6, 'a': 7, 'p': 8, 'o': 9, 'd': 10, 'f': 11, 'l': 12, 'w': 13, 'r': 14, 'h': 15, "'": 16, 'g': 17, 'm': 18, 't': 19, 'k': 20, '.': 21, 'n': 22, 'b': 23, 'i': 24}


In [0]:
#hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10 #Any arbitary number
learning_rate = 0.1

In [9]:
print(dic_size)
print(x_data)
print(x_one_hot)

25
[[24, 11, 5, 6, 9, 0, 5, 13, 7, 22], [11, 5, 6, 9, 0, 5, 13, 7, 22, 19], [5, 6, 9, 0, 5, 13, 7, 22, 19, 5], [6, 9, 0, 5, 13, 7, 22, 19, 5, 19], [9, 0, 5, 13, 7, 22, 19, 5, 19, 9], [0, 5, 13, 7, 22, 19, 5, 19, 9, 5], [5, 13, 7, 22, 19, 5, 19, 9, 5, 23], [13, 7, 22, 19, 5, 19, 9, 5, 23, 0], [7, 22, 19, 5, 19, 9, 5, 23, 0, 24], [22, 19, 5, 19, 9, 5, 23, 0, 24, 12], [19, 5, 19, 9, 5, 23, 0, 24, 12, 10], [5, 19, 9, 5, 23, 0, 24, 12, 10, 5], [19, 9, 5, 23, 0, 24, 12, 10, 5, 7], [9, 5, 23, 0, 24, 12, 10, 5, 7, 5], [5, 23, 0, 24, 12, 10, 5, 7, 5, 1], [23, 0, 24, 12, 10, 5, 7, 5, 1, 15], [0, 24, 12, 10, 5, 7, 5, 1, 15, 24], [24, 12, 10, 5, 7, 5, 1, 15, 24, 8], [12, 10, 5, 7, 5, 1, 15, 24, 8, 3], [10, 5, 7, 5, 1, 15, 24, 8, 3, 5], [5, 7, 5, 1, 15, 24, 8, 3, 5, 10], [7, 5, 1, 15, 24, 8, 3, 5, 10, 9], [5, 1, 15, 24, 8, 3, 5, 10, 9, 22], [1, 15, 24, 8, 3, 5, 10, 9, 22, 16], [15, 24, 8, 3, 5, 10, 9, 22, 16, 19], [24, 8, 3, 5, 10, 9, 22, 16, 19, 5], [8, 3, 5, 10, 9, 22, 16, 19, 5, 10], [3, 5, 10, 

In [6]:
#data setting
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+1+sequence_length]
  print(i, x_str, '->', y_str)

  x_data.append([char_dic[c] for c in x_str])
  y_data.append([char_dic[c] for c in y_str])

x_one_hot = [np.eye(dic_size)[x] for x in x_data]


0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [0]:
#transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [0]:
#decclare RNN + FC
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

net = Net(dic_size, hidden_size, 2)

In [0]:
#loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [16]:
#start training
for i in range(100):
  optimizer.zero_grad()
  outputs = net(X)
  loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  results = outputs.argmax(axis=2)
  predict_str = ""
  #여기서 j는 sequence당 results의 인덱스, 하나의 시퀀스 내용 result
  for j, result in enumerate(results):
    if j == 0:
      predict_str = ''.join([char_set[t] for t in result])
    else:
      predict_str += char_set[result[-1]]
  print(predict_str, loss.item())

p you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea. 0.24382713437080383
l you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea. 0.24368861317634583
p you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea. 0.24323910474777222
l you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea. 0.24278675019741058
g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea. 0.2423931360244751
g

In [17]:
print(results)

tensor([[12,  5,  6,  ...,  7, 22, 19],
        [ 9,  6,  9,  ..., 22, 19,  5],
        [19,  9,  0,  ..., 19,  5, 19],
        ...,
        [ 9,  9, 11,  ...,  5,  1,  2],
        [19, 11,  5,  ...,  1,  2,  7],
        [22,  5, 19,  ...,  2,  7, 21]], grad_fn=<NotImplemented>)
